In [10]:
from fastai.imports import *
from fastai.structured import *

from fastai.metrics import *
import torch.nn as nn
from fastai.model import *
from fastai.dataset import *

import numpy as np
import pandas as pd
df = pd.read_csv('data/transform_hot_full.csv', index_col=0)

### General helper functions

In [11]:
m_names = pd.read_csv("exports/game_map_cats.csv",header=None, index_col=0, squeeze=True).to_dict()
map_names = [n for x, n, in m_names.items()]
map_names_swapped = dict([(value, key) for key, value in m_names.items()])

h_names= pd.read_csv("exports/hero_name_cats.csv",header=None, index_col=0, squeeze=True).to_dict()
hero_names = [n for x, n, in h_names.items()]

In [12]:
mns = [x[1] for x in list(enumerate(map_names))]
yhns = ['yours_'+x[1] for x in list(enumerate(hero_names))]
thns = ['theirs_'+x[1] for x in list(enumerate(hero_names))]
col_names =mns+yhns+thns

def hero_one_full(game_map = '', winners = [], losers = []):
    new_row = pd.Series(index = col_names, dtype = 'boolean')
    new_row[:] = False
    for x in hero_names:
        if x in winners:
            new_row['yours_'+x] = True
        if x in losers:
            new_row['theirs_'+x] = True
    new_row[game_map] = True
    return new_row

def hero_hot_mini(game_map = '', winners = [], losers = []):
    new_row = pd.Series(index = hero_names, dtype = 'int8')
    for x in hero_names:
        if x in winners:
            new_row[x] = 2
        if x in losers:
            new_row[x] = 1
    game_map_index = int(map_names_swapped[game_map])
    new_row = new_row.append(pd.Series(game_map_index, index = ['game_map'], dtype = 'int8'))
    return new_row

def hero_one_hot(game_map = '', winners = [], losers = []):
    return hero_one_full(game_map = game_map, winners = winners, losers = losers)

In [17]:
def split_random_masks(a, n1, n2):
    msk = np.random.rand(len(a)) < (n1 + n2)
    msk1 = ((np.random.rand(len(a)) < n1/(n1+n2)) & msk)
    msk2 = (~msk1 & msk)
    return ~msk, msk1, msk2

### NN Helper methods

In [18]:
def show(img, title=None):
    plt.imshow(img, cmap='gray');
    if title is not None: plt.title(title)

In [19]:
def plots(ims, figsize=(12,6), rows=2, titles=None):
    f = plt.figure(figsize = figsize)
    cols = len(ims)//rows
    for i in range(len(ims)):
        sp = f.add_subplot(rows, cols, i+1)
        sp.axis('Off')
        if titles is not None: sp.set_title(titles[i], fontsize=16)
        plt.imshow(ims[i], cmap='gray')

In [20]:
#?plt.imshow

##### Split datasets

In [21]:
SAMPLE_SIZE = 5000
data1 = df.sample(n = SAMPLE_SIZE, random_state = 1, axis = 0)
df_trn, y_trn, nas = proc_df(data1, 'outcome')
val_ratio = 0.2
tst_ratio = 0.1
#y_trn=y_trn.astype('bool')
#n_trn = int(len(df_trn) * train_required_ratio)
tr_mask, val_mask, test_mask = split_random_masks(df_trn, val_ratio, tst_ratio)
x_train = df_trn[tr_mask].copy()
y_train = y_trn[tr_mask].copy()
x_valid = df_trn[tr_mask].copy()
y_valid = y_trn[tr_mask].copy()
x_test = df_trn[tr_mask].copy()
y_test = y_trn[tr_mask].copy()

print("train: ",x_train.shape, "  val:", x_valid.shape, " tst: ", x_test.shape)
apply_cats(x_valid, x_train)

train:  (3509, 188)   val: (3509, 188)  tst:  (3509, 188)


## Plots

In [22]:
x_valid.shape

(3509, 188)

In [24]:
#x_imgs = np.reshape(x_valid, (-1,28,28)); x_imgs.shape

In [ ]:
#show(x_imgs[0,10:15,10:15])

In [ ]:
#plots(x_imgs[:8], titles=y_valid[:8])

# Neural Net for Logistic Regression in PyTorch

so this error is due to a bad python version, in 3.7 async is now a keyword..., need 3.6 python version to run this old fastai

In [26]:
net = nn.Sequential(
    nn.Linear(1000, 200),
    nn.LogSoftmax(),
    nn.Linear(200, 10),
    nn.LogSoftmax()
).cuda()
#this is a two layer neural net
#.cuda tells to run on GPU, if didnt say that, would run on CPU

In [30]:
#mp = ImageClasifireData.from_arrays(path, (x, y), (x_valid, y_valid))

In [29]:
loss = nn.NLLLoss() #negative low likelyhood loss = cross entropy. Either binary or categorical
metrics=[accuracy]
opt=optim.Adam(net.parameters())

### Fitting the model

In [33]:
?ModelData

In [ ]:
fit(net, md, epochs=1, crit=loss, opt=opt, metrics=metrics) #pythorch uses word criterion instead of loss

In [ ]:
preds = predict(net, md.val_dl)
preds.shape

In [ ]:
preds.argmax(axis=1)[:5] #argmax > figures out on this axis (for 10 in this case), return the index of the max value

In [ ]:
preds = preds.argmax(1) #saving this

In [ ]:
np.mean(preds == y_valid)

In [ ]:
plots(x_imgs[:8], titles=preds[:8])

#we have just built a logistic regression, it is not a deep neural net yet

In [ ]:
def get_weights(dims): return nn.Parameter(torch.randn(dims)/dims[0]) # because there are many layers, so need to make sure that the mean inputs not gonna change, otherwise its gonna diverge infinitely or converge it the number is bigger (gradient explosion)
#google: kaiming he initialization

In [ ]:
class LogReg(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1_w = get_weights(28*28, 10) # Layer 1 weights
        self.l1_b = get_weights(10)   # Layer 1 bias
    def forward(self, x): #in pytorch it has special meaning, it will get called when the layer gets calculated, its gonna get passed data form previous layer
        x = x.view(x.size(0), -1) #view = reshape
        x = torch.matmul(x, self.l1_w) +self.l1_b # Linear layer
        x = torch.log(torch.exp(x)/(torch.exp(x).sum(dim=0))) #this is a softmax (because we want all probabiliteis of outcomes to sumup to 1), so it behaves so tthat it returns something that behaves as probabilities
        return x
#there is a lot of info on pytorch website on how to create tensors and modify stuff

In [ ]:
#we define a * x, which are weights, then
#+b is called bias

In [ ]:
net2 = LogReg().cuda()
opt=optim.Adam(net2.parameters())

In [ ]:
fit(net2, md, epochs=1, crit=loss, opt=opt, metrics=metrics)

In [ ]:
1:19:30